In [2]:
import os
import csv
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [154]:
df=pd.read_csv("./train.tsv",header=None, usecols=[1,3],names=['humor', 'text'])

In [155]:
df_humor=df[df['humor']==1]

In [156]:
searchfor=['deleted','removed']

In [157]:
df_humor=df_humor[~df_humor["text"].str.contains('|'.join(searchfor))]

In [158]:
df_humor['User']=" "
df_humor['Humorous reply']=" "

In [159]:
df_humor['User']=df_humor['text'].apply(lambda x:x.split('_____')[0])

In [160]:
df_humor['Humorous reply']=df_humor['text'].apply(lambda x:x.split('_____')[1])

In [161]:
df_humor

,humor,text,User,Humorous reply
2,1,What do you call a cow with 3 legs?_____Lean b...,What do you call a cow with 3 legs?,Lean beef.What do you call a cow with no legs?...
4,1,"In a crowded city at a bus stop, a beautiful y...","In a crowded city at a bus stop, a beautiful y...",As the bus rolled up and it became her turn to...
5,1,My neighbor knocked on my door this morning at...,My neighbor knocked on my door this morning at...,Luckily for him i was still up playing my drums
8,1,Little Bobby was running through the woods____...,Little Bobby was running through the woods,Little Bobby was running through the woods beh...
9,1,"Whenever I’m at the therapist’s waiting room, ...","Whenever I’m at the therapist’s waiting room, ...","Everyone hates it, but I’m a fan."
...,...,...,...,...
19428,1,"My friend just asked me, ""If you became invisi...","My friend just asked me, ""If you became invisi...","I said, ""I'd go to Paris, find a performing st..."
19429,1,"How many ""friend-zones"" guys does it take to c...","How many ""friend-zones"" guys does it take to c...","None, they'll just compliment it and get pisse..."
19435,1,"If you pronounce ""fuck off"" backwards._____you...","If you pronounce ""fuck off"" backwards.",you say it in a British accent.&#x200B;Edit:Th...
19436,1,A priest has a heart attack..._____A priest ha...,A priest has a heart attack...,A priest has a heart attack and is rushed to t...


In [162]:
df_final=df_humor.drop(columns=['text','humor'])
df_final=df_final[df_final['Humorous reply'].str.len() <= 300]

In [163]:
df2=pd.read_csv('./ocr.csv')

In [164]:
df2=df2.drop(columns=['humor','com_score'])
df2.rename(columns={'U':'User','Humorous_reply':'Humorous reply'},inplace = True)

In [165]:
df2

,User,Humorous reply
0,WHY ARE YOU TALKING IN THE MIDDLE OF MY CLASS?,WHY ARE YOU TEACHING IN THE MIDDLE OF MY CONVE...
1,Hey long time no talk,let's keep it that way
2,what happens to the car if you press the brake...,it takes a screenshot
3,WHAT EVENT IS MADONNA ATTENDING HERE?,PARENT TEACHER NIGHT
4,"My friend : Bro, ""your mom"" jokes are SO overused","Me saying ""just like your mom"""
...,...,...
311,I’m in a lot of pain and could use some light ...,Do you know the one about the blind hooker?
312,I’m going out tomorrow night with some friends...,new ice age?
313,Why do you pick someone up off the ground when...,i guess i get a little carried away.
314,What's your unique talent?,"I've had people start looking around, trying t..."


In [166]:
df_final=df_final.append(df2)
df_final

,User,Humorous reply
2,What do you call a cow with 3 legs?,Lean beef.What do you call a cow with no legs?...
5,My neighbor knocked on my door this morning at...,Luckily for him i was still up playing my drums
9,"Whenever I’m at the therapist’s waiting room, ...","Everyone hates it, but I’m a fan."
11,"A lady died and went to heaven, upon seeing Go...","“Ok, ask away” God said. “Do vaccines cause au..."
16,My girlfriend dressed up as a policewoman and ...,After 2 minutes all charges were dropped due t...
...,...,...
311,I’m in a lot of pain and could use some light ...,Do you know the one about the blind hooker?
312,I’m going out tomorrow night with some friends...,new ice age?
313,Why do you pick someone up off the ground when...,i guess i get a little carried away.
314,What's your unique talent?,"I've had people start looking around, trying t..."


In [167]:
df_final.to_csv('./final.csv')

In [121]:
df_final=pd.read_csv('./final.csv')
df_final=df_final[df_final['Humorous reply']!=' ']
df_final=df_final.drop_duplicates(subset=['User'])

df_final['User']=df_final['User'].apply(lambda x:"User: "+x)
df_final['Humorous reply']=df_final['Humorous reply'].apply(lambda x:"Humorous reply: "+x)

In [122]:
df_final

,Unnamed: 0,User,Humorous reply
0,2,User: What do you call a cow with 3 legs?,Humorous reply: Lean beef.What do you call a c...
1,5,User: My neighbor knocked on my door this morn...,Humorous reply: Luckily for him i was still up...
2,9,User: Whenever I’m at the therapist’s waiting ...,"Humorous reply: Everyone hates it, but I’m a fan."
3,11,"User: A lady died and went to heaven, upon see...","Humorous reply: “Ok, ask away” God said. “Do v..."
4,16,User: My girlfriend dressed up as a policewoma...,Humorous reply: After 2 minutes all charges we...
...,...,...,...
6931,311,User: I’m in a lot of pain and could use some ...,Humorous reply: Do you know the one about the ...
6932,312,User: I’m going out tomorrow night with some f...,Humorous reply: new ice age?
6933,313,User: Why do you pick someone up off the groun...,Humorous reply: i guess i get a little carried...
6934,314,User: What's your unique talent?,Humorous reply: I've had people start looking ...


In [123]:
my_data=df_final

In [124]:
train_size = 0.7
validate_size = 0.2
train, validate, test = np.split(my_data.sample(frac=1), [int(train_size * len(my_data)), int((validate_size + train_size) * len(my_data))])

In [127]:
train

,Unnamed: 0,User,Humorous reply
292,860,User: Men think about sex every 7 seconds...,Humorous reply: Which is why I eat hotdogs in ...
3556,10334,User: Anyone need a slutty costume for Halloween?,Humorous reply: Just dress up as one of my pro...
744,2105,User: What’s the difference between outlaws an...,Humorous reply: Outlaws are wanted.
376,1091,User: My friend Dave drowned.,Humorous reply: At his funeral we got him a wr...
895,2562,User: As I slipped my finger slowly inside her...,"Humorous reply: I slid my finger back out, and..."
...,...,...,...
756,2138,User: Why will congress never impeach Trump?,Humorous reply: Republicans always insist on c...
1155,3389,"User: At age 4, success is not peeing in your ...","Humorous reply: At age 50, success is having m..."
494,1450,User: If you could exterminate any race what w...,"Humorous reply: Personally, I‘d get rid of the..."
6907,287,User: A friend of mine invented a sex position...,Humorous reply: Fucking savage.


In [128]:
validate

,Unnamed: 0,User,Humorous reply
806,2298,User: How many introverts does it take to chan...,Humorous reply: Why must it be a group activity?
2442,7019,"User: So, a snake walks into a bar.","Humorous reply: The bartender asks, “how did y..."
1764,5106,User: Can a woman make her husband a millionaire?,"Humorous reply: Of course, if he’s a billionai..."
699,2012,User: I was walking home last night and decide...,Humorous reply: 3 girls walked up to me and ex...
1364,3986,User: I ordered a chicken and an egg from Amaz...,Humorous reply: I'll let you know.
...,...,...,...
2541,7311,User: What's a pirate's least favorite letter?,"Humorous reply: Dear sir,Your internet access ..."
458,1355,User: My dad died this day last year because w...,Humorous reply: As he was dying he kept saying...
2450,7039,User: Making love for the first time,Humorous reply: Before my girlfriend and i mad...
629,1820,User: I just realized my wife left me because ...,"Humorous reply: Oh well, hindsight is 1."


In [129]:
test

,Unnamed: 0,User,Humorous reply
6910,290,User: Did my dad just sleepwalk?!,Humorous reply: >my dad is sleeping to the bed...
485,1431,User: A wife got so mad at her husband she pac...,Humorous reply: As he walked to the door she y...
6659,39,User: how would you feel if a girl gave YOU a ...,Humorous reply: I would just be glad they came...
1567,4531,User: I'm so straight I don't touch myself whe...,Humorous reply: My buddy Brian does it for me
1366,4000,User: A man was sunbathing naked at the beach.,"Humorous reply: For the sake of civility, and ..."
...,...,...,...
1334,3905,"User: I remember when I was a kid, I could go ...","Humorous reply: Nowadays, they got cameras eve..."
497,1460,User: An elderly couple is in church. The wife...,"Humorous reply: The husband says, ""Change the ..."
2124,6205,"User: My brother just updated his status to ""I...",Humorous reply: I always knew he liked them y...
2193,6386,User: Wearing crocs is like getting a blowjob ...,Humorous reply: It feels great until you look ...


In [130]:
file1 = open("train.txt","a")

In [131]:
for index, row in train.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file1.write(str1)
    file1.write('\n')
    file1.write(str2)
    file1.write('<|endoftext|>')

In [132]:
file1.close

<function TextIOWrapper.close()>

In [133]:
file2 = open("val.txt","a")

In [134]:

for index, row in validate.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file2.write(str1)
    file2.write('\n')
    file2.write(str2)
    file2.write('<|endoftext|>')



In [135]:
file2.close

<function TextIOWrapper.close()>

In [136]:
file3 = open("test.txt","a")
for index, row in test.iterrows():
    str1=row['User']
    str2=row['Humorous reply']
    file3.write(str1)
    file3.write('\n')
    file3.write(str2)
    file3.write('<|endoftext|>')



In [137]:
file3.close

<function TextIOWrapper.close()>

In [109]:
test

,Unnamed: 0,User,Humorous reply
146,454,User: What did Lochte say after his teammates ...,"Humorous reply: ""...and I would have gotten aw..."
6817,197,User: Four hours later and 45 minutes after la...,Humorous reply: this is a good way to pass tim...
5460,15947,"User: Color vs Colour, Favorite vs Favourite, ...",Humorous reply: British English: I think you'r...
4973,14479,User: MOVIE RATINGS EXPLAINED,Humorous reply: G: Nobody gets the girl.PG: Th...
5205,15166,"User: My uncle's zodiac sign was Cancer, which...",Humorous reply: from being crushed by a giant ...
...,...,...,...
669,1920,"User: Color vs Colour, Favorite vs Favourite, ...",Humorous reply: British English: I think you'r...
5488,16024,User: A man was arrested and taken to an inter...,"Humorous reply: He says to the police officer,..."
6011,17539,User: When I drink too much alcohol I’m called...,Humorous reply: but when I smoke a lot of weed...
599,1725,User: Women are actually turning into good dri...,"Humorous reply: So if you’re a good driver, wa..."


In [138]:
file4 = open("test_user.txt","a")
for index, row in test.iterrows():
    str1=row['User']
#     str2=row['Humorous reply']
    file4.write(str1)
    file4.write('\n')
#     file4.write('Humorous reply: ')
file4.close    

<function TextIOWrapper.close()>